#### Add Survey Data to Ontology

In [21]:
import pandas as pd
from rdflib import Graph, Namespace, Literal, URIRef, RDF

In [22]:
xlsx_file_path = 'C:/Users/ricar/Desktop/Data/survey_data_for_ontology.xlsx'
df = pd.read_excel(xlsx_file_path)

In [23]:
# define namespace
graph = Graph()
p_ns = Namespace("https://w3id.org/survey-ontology#") #participant

In [24]:
# Iterate over each row in csv file
for index, row in df.iterrows():
    # Create individual for each participant
    participant_uri = p_ns[f'participant{row["Participant"]}']
    graph.add((participant_uri, RDF.type, p_ns.Participant))

    # Add ID as data property assertion
    graph.add((participant_uri, p_ns.hasID, Literal(row['Participant'])))

    # Create individual for each SurveyCompletionTask
    sct_uri = p_ns[f'SurveyCompletionTask{index + 1}']
    graph.add((sct_uri, RDF.type, p_ns.SurveyCompletionTask))

    # Link the participant to the SurveyCompletionTask
    graph.add((sct_uri, p_ns.completedFor, participant_uri))

    # Iterate through each column in csv file
    for column in df.columns:
        # Ignore column 'lfdn'
        if column != 'lfdn':
            # Create individual for CompletedQuestion
            cq_uri = p_ns[f'Completed.{column}.{index + 1}']
            graph.add((cq_uri, RDF.type, p_ns.CompletedQuestion))

            # Link CompletedQuestion with SurveyCompletionTask
            graph.add((cq_uri, p_ns.answeredIn, sct_uri))

            # Link CompletedQuestion with corresponding question
            question_uri = p_ns[f'{column}']
            graph.add((cq_uri, p_ns.completesQuestion, question_uri))

            # Check if string
            if isinstance(row[column], float):
                # If it is a number, add it as an answer
                graph.add((cq_uri, p_ns.hasAnswer, Literal(row[column])))
            elif isinstance(row[column], str):
                # Add answer
                graph.add((cq_uri, p_ns.hasAnswer, Literal(row[column])))
            else:
                # Mappe numerical values into text
                answer_mapping = {
                    1: 'StronglyDisagree',
                    2: 'Disagree',
                    3: 'MoreOrLessDisagree',
                    4: 'Undecided',
                    5: 'MoreOrLessAgree',
                    6: 'Agree',
                    7: 'StronglyAgree'
                }

                # Add answers
                answer_text = answer_mapping.get(row[column])
                graph.add((cq_uri, p_ns.hasAnswer, Literal(answer_text)))

In [20]:
# Save RDF-Graph
output_file_path = 'C:/Users/ricar/Desktop/Data/survey_ontology_data.ttl'
graph.serialize(output_file_path, format='turtle')

<Graph identifier=Nf0d26cf7ef92429f8bcdc6c9c9a24236 (<class 'rdflib.graph.Graph'>)>